**Второй подход (разница в методе oversampling'а):**

- используем отфильтрованный трейн
- делаем oversampling при помощи SMOTE
- запускаем gridsearch для catboost c cv=3 и scoring='roc_auc'
- фильтруем тест через csv со списком признаков после фильтрации 
- запускаем predict_proba
- считаем метрики
- закидываем submission_version_2

Импортируем библиотеки

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

In [3]:
VERSION = 2

Считываем отфильтрованные данные

In [4]:
X = pd.read_parquet("../../data/intermediate_data/filter_train.parquet")

In [5]:
print(X.shape)

(519615, 228)


In [6]:
X.head()

,feature547,feature418,feature867,feature15,feature641,feature734,feature549,feature674,feature442,feature763,...,feature717,feature844,feature380,feature550,feature864,feature727,feature454,feature369,feature367,feature409
0,37,77,7063,6,0,14,17,1,35,0,...,4,1283,0,49,1619,28,0,1,4,56
1,1,77,7063,135,0,14,17,0,35,153,...,4,1283,0,49,2092,28,6,1,4,56
2,37,77,7063,0,0,14,17,0,35,153,...,4,1283,0,49,7174,28,83,1,3,56
3,37,77,7063,0,0,14,17,0,35,1,...,4,1283,0,49,7174,28,0,1,2,56
4,37,77,7063,0,0,14,17,0,35,153,...,4,1283,0,49,1439,28,1,2,2,56


In [7]:
X.target.value_counts()

target
0    501078
1     18537
Name: count, dtype: int64

In [8]:
y = X["target"]
X.drop(columns = ["target", "id"], inplace = True)

In [9]:
y.tail(3)

519612    0
519613    0
519614    1
Name: target, dtype: int64

Делаем oversampling через SMOTE

In [10]:
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

In [11]:
X_resampled.shape

(1002156, 226)

Создаем сетку параметров для CatBoost и запускаем перебор параметров с CV

In [12]:
catboost_parameters = {
    'depth'         : [4, 7, 10],
    'learning_rate' : [0.01, 0.03, 0.05],
    'iterations'    : [50, 250]
}

Разделяем данные на обучающую и тестовую выборки

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,
                                                    y_resampled, 
                                                    test_size=0.2, 
                                                    stratify=y_resampled, 
                                                    random_state=23)

In [14]:
grid_search_ct = GridSearchCV(estimator=CatBoostClassifier(),
                              param_grid=catboost_parameters,
                              scoring='roc_auc',
                              cv=3,
                              n_jobs=5,
                              verbose=2)

In [15]:
grid_search_ct.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
0:	learn: 0.6758504	total: 266ms	remaining: 13s
1:	learn: 0.6601715	total: 474ms	remaining: 11.4s
2:	learn: 0.6448022	total: 634ms	remaining: 9.94s
3:	learn: 0.6316364	total: 735ms	remaining: 8.46s
4:	learn: 0.6202636	total: 839ms	remaining: 7.55s
5:	learn: 0.6105657	total: 954ms	remaining: 7s
6:	learn: 0.5946267	total: 1.1s	remaining: 6.75s
7:	learn: 0.5772276	total: 1.2s	remaining: 6.28s
8:	learn: 0.5690674	total: 1.32s	remaining: 6.03s
9:	learn: 0.5533364	total: 1.45s	remaining: 5.8s
10:	learn: 0.5435953	total: 1.58s	remaining: 5.62s
11:	learn: 0.5357673	total: 1.68s	remaining: 5.33s
12:	learn: 0.5247164	total: 1.83s	remaining: 5.22s
13:	learn: 0.5120095	total: 2.01s	remaining: 5.16s
14:	learn: 0.5062349	total: 2.15s	remaining: 5.02s
15:	learn: 0.4955700	total: 2.33s	remaining: 4.95s
16:	learn: 0.4878732	total: 2.51s	remaining: 4.88s
17:	learn: 0.4819781	total: 2.67s	remaining: 4.75s
18:	learn: 0.4734600	total: 2.84s	remai

/home/jupyter-user/andreeva.dal/infoletters/venv/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


58:	learn: 0.2747791	total: 8.82s	remaining: 28.5s
59:	learn: 0.2719822	total: 8.94s	remaining: 28.3s
60:	learn: 0.2688091	total: 9.24s	remaining: 28.6s
61:	learn: 0.2669910	total: 9.37s	remaining: 28.4s
62:	learn: 0.2645811	total: 9.54s	remaining: 28.3s
63:	learn: 0.2623712	total: 9.66s	remaining: 28.1s
64:	learn: 0.2602536	total: 9.79s	remaining: 27.9s
65:	learn: 0.2585259	total: 9.96s	remaining: 27.8s
66:	learn: 0.2555989	total: 10.1s	remaining: 27.6s
67:	learn: 0.2528410	total: 10.2s	remaining: 27.4s
68:	learn: 0.2509150	total: 10.4s	remaining: 27.2s
69:	learn: 0.2478803	total: 10.5s	remaining: 27.1s
70:	learn: 0.2458496	total: 10.7s	remaining: 26.9s
71:	learn: 0.2435173	total: 10.8s	remaining: 26.7s
72:	learn: 0.2408580	total: 11s	remaining: 26.7s
73:	learn: 0.2394489	total: 11.1s	remaining: 26.5s
74:	learn: 0.2369255	total: 11.3s	remaining: 26.4s
75:	learn: 0.2342543	total: 11.5s	remaining: 26.3s
76:	learn: 0.2322012	total: 11.6s	remaining: 26.1s
77:	learn: 0.2309000	total: 11.7s

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x7ff5bec76190>,
             n_jobs=5,
             param_grid={'depth': [4, 7, 10], 'iterations': [50, 250],
                         'learning_rate': [0.01, 0.03, 0.05]},
             scoring='roc_auc', verbose=2)

In [16]:
print("the best estimator:\n", grid_search_ct.best_estimator_)
print("the best score:\n", grid_search_ct.best_score_)
print("the best parameters:\n", grid_search_ct.best_params_)

the best estimator:
the best score:
 0.9885612794486578
the best parameters:
 {'depth': 10, 'iterations': 250, 'learning_rate': 0.05}


In [17]:
pred = grid_search_ct.predict_proba(X_test)
pred = pred[:, 1]

In [18]:
for threshold in [0.1, 0.3, 0.5]:
    pred_binary = (pred >= threshold)
    
    print("threshold:", threshold)
    print("F1_SCORE:", f1_score(y_test, pred_binary))
    print("PRECISION:", precision_score(y_test, pred_binary))
    print("RECALL:", recall_score(y_test, pred_binary))
    print("ROC_AUC:", roc_auc_score(y_test, pred))
    print()

threshold: 0.1
F1_SCORE: 0.9508710291318987
PRECISION: 0.9304486506102336
RECALL: 0.9722100263430989
ROC_AUC: 0.9884187227513214

threshold: 0.3
F1_SCORE: 0.9795125218167796
PRECISION: 0.9963769611890999
RECALL: 0.9632094675500918
ROC_AUC: 0.9884187227513214

threshold: 0.5
F1_SCORE: 0.980013531180214
PRECISION: 0.9995849201490137
RECALL: 0.9611938213458928
ROC_AUC: 0.9884187227513214



In [20]:
best_threshold = 0.1

Сохраняем submission

In [21]:
X_test_submit = pd.read_parquet("../../data/input_data/test_sber.parquet")
#X.drop(columns = ["sample_ml_new", "id"], inplace = True)

In [22]:
print(X_test_submit.shape)

(173433, 1078)


In [23]:
X_test_submit.head()

,id,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,3,3,1696,458,26,102479,22,16,0,121,...,779,7740,9577,254,0,355,308,779,7740,9577
1,4,3,1688,53,78,103922,191,64,0,0,...,79401,109240,153820,24766,48600,46029,65113,79401,109240,153820
2,12,3,1689,13,81,104111,191,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,16,3,1761,1759,44,102433,191,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,3,1761,1759,77,102010,191,34,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
filtered_columns = pd.read_csv("../../data/intermediate_data/cols_after_preprocessing.csv")

In [25]:
cols = list(set(filtered_columns.cols_after_preprocessing.values) & set(X_test_submit.columns))

In [26]:
X_filtered = X_test_submit[cols]

In [28]:
X_filtered.drop(columns=["id"], inplace=True)

/tmp/ipykernel_30536/46465938.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_filtered.drop(columns=["id"], inplace=True)


In [29]:
print(X_filtered.shape)

(173433, 226)


In [30]:
X_filtered.head()

,feature182,feature322,feature32,feature831,feature541,feature776,feature422,feature397,feature417,feature362,...,feature558,feature421,feature506,feature488,feature529,feature416,feature414,feature717,feature736,feature584
0,106508,46,163,1306,2,92,56,21,50,18,...,20,49,56,58,0,34,101,4,12,0
1,326142,35,1756,0,1,92,56,21,50,18,...,20,49,40,75,7,34,101,4,0,0
2,431252,27,1756,1306,0,92,56,21,50,18,...,20,49,143,142,3,34,101,4,0,0
3,178611,117,1756,1306,50,92,56,21,50,18,...,20,49,190,115,360,34,101,4,12,0
4,169414,704,1756,1306,0,92,56,21,50,18,...,20,49,277,165,0,34,101,4,12,0


In [31]:
pred = grid_search_ct.predict_proba(X_filtered)

pred = pred[:, 1]
pred_binary = (pred >= best_threshold).astype(int)

In [32]:
submission = pd.read_csv("../../data/intermediate_data/sample_submission.csv")
submission.head(10)

,id,target_bin,target_prob
0,3,0,0.03
1,4,0,0.03
2,12,1,0.03
3,16,1,0.03
4,20,0,0.03
5,23,0,0.03
6,26,0,0.03
7,50,0,0.03
8,51,1,0.03
9,53,0,0.03


In [33]:
submission["target_prob"] = pred
submission["target_bin"] = pred_binary
submission.head(10)

,id,target_bin,target_prob
0,3,0,0.054599
1,4,0,0.048016
2,12,0,0.015607
3,16,0,0.019199
4,20,0,0.057165
5,23,0,0.017055
6,26,0,0.029891
7,50,1,0.110303
8,51,0,0.028012
9,53,0,0.013604


In [ ]:
submission.to_csv(f"../../data/output_data/submission_version_{VERSION}_{best_threshold}.csv", index=False)

---